# 차종인식 모델

- 데이터셋 출처: AIHUB의 '차량 외관 영상 데이터' (전체 크기 압축되어 약 80GB)
  - https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=554
- 사용 데이터 셋: 현대 + 기아의 자료만 사용 (현 환경에서 전체 데이터 사용 불가)
- 자료 정리는 Local(Macbook)에서 진행하고, 모델 학습은 Colab에서 진행 예정

### Model 동작 방식
1. 파트별로 객체탐지 모델
2. Crop 된 이미지를 이용하여 색상감지 모델
3. Crop 된 이미지를 이용하여 모델/연식 등 분류 모델

### Model 1 (Part 검출 모델)

In [1]:
import time
import os
import json
from shutil import copyfile
import yaml

from timeit import default_timer as timer
import pandas as pd
import torch

In [ ]:
part = { "P00":"차량전체", "P01":"프론트범퍼", "P02":"리어범퍼", "P03":"타이어(휠)", "P04":"A필러",
       "P05":"C필러", "P06":"사이드미러", "P07":"앞도어", "P08":"뒷도어", "P09":"라디에이터그릴",
       "P10":"헤드램프", "P11":"리어램프", "P12":"보닛", "P13":"트렁크", "P14":"루프", }

In [ ]:
def copy_yolo_data(src, dst):
    filenames = os.listdir(src)

    for filename in filenames:
        if os.path.isdir(os.path.join(src, filename)):
            copy_yolo_data(os.path.join(src, filename), dst)
            
        if os.path.isfile(os.path.join(src, filename)):
            if filename != '.DS_Store':
                if os.path.splitext(filename)[1] == '.json':
                    copyfile(os.path.join(src, filename),os.path.join(dst+'/annotations', filename))
                elif os.path.splitext(filename)[1] == '.jpg':
                    copyfile(os.path.join(src, filename),os.path.join(dst+'/images', filename))

In [ ]:
# Train 데이터를 YOLOv5 형식에 맞도록 작업하기 위하여 Copy

start = time.time()
copy_yolo_data('/Users/okchuri/project/codestates/CP2/Data/train', '/Users/okchuri/project/codestates/CP2/YOLO_Data/train')    
print("time :", time.time() - start)

In [ ]:
# Val 데이터를 YOLOv5 형식에 맞도록 작업하기 위하여 Copy

start = time.time()
copy_yolo_data('/Users/okchuri/project/codestates/CP2/Data/validation', '/Users/okchuri/project/codestates/CP2/YOLO_Data/validation')    
print("time :", time.time() - start)

In [ ]:
def make_part_label(src, dst):
    with open(src, 'r') as jess:
        jess_dict = json.load(jess)
    
    resolution = jess_dict['rawDataInfo']['resolution']
    width, height = int(resolution.split('*')[0]), int(resolution.split('*')[1])
    
    with open(dst, 'w') as f:
        for obj in jess_dict['learningDataInfo']['objects']:
            classID = obj['classId'].split('.')[0]
            cx = (int(obj['coords']['tl']['x']) + int(obj['coords']['tr']['x'])) / 2 / width
            cy = (int(obj['coords']['tl']['y']) + int(obj['coords']['bl']['y'])) / 2 / height
            w = (int(obj['coords']['tr']['x']) - int(obj['coords']['tl']['x'])) / width
            h = (int(obj['coords']['bl']['y']) - int(obj['coords']['tl']['y'])) / height
            
            txt = str(classID) + ' ' + str(cx) + ' ' + str(cy) + ' ' + str(w) + ' ' + str(h) + '\n'
            
            f.write(txt)
            
def make_part_labels(src, dst):
    filenames = os.listdir(src)

    for filename in filenames:
        if filename != '.DS_Store':
            if os.path.splitext(filename)[1] == '.json':
                make_part_label(os.path.join(src, filename), dst+os.path.splitext(filename)[0]+'.txt')

In [ ]:
start = time.time()
make_part_labels('/Users/okchuri/project/codestates/CP2/YOLO_Data/train/annotations/', '/Users/okchuri/project/codestates/CP2/YOLO_Data/train/labels/')
print("time :", time.time() - start)

In [ ]:
start = time.time()
make_part_labels('/Users/okchuri/project/codestates/CP2/YOLO_Data/validation/annotations/', '/Users/okchuri/project/codestates/CP2/YOLO_Data/validation/labels/')
print("time :", time.time() - start)

In [ ]:
classes = ['P00', 'P01', 'P02', 'P03', 'P04', 'P05', 'P06', 'P07',
          'P08', 'P09', 'P10', 'P11', 'P12', 'P13', 'P14', 'P15']

In [ ]:
yaml_file = 'car_part.yaml'

yaml_data = dict(
    path = "data",
    train = "train",
    val = "val",
    nc = len(classes),
    names = classes
)

with open(yaml_file, 'w') as f:
    yaml.dump(yaml_data, f, explicit_start = True, default_flow_style = False)

### Model 2 (모델명, 년식, 색상: 485 classes) <-- (모델명, 년식, 색상, 트림: 1199 classes)

In [4]:
model_classes = []

def get_model_class(src):
    with open(src, 'r') as jess:
        jess_dict = json.load(jess)
        
    rawDataID = jess_dict['rawDataInfo']['rawDataID'][9:21]
    if not rawDataID in model_classes:
        model_classes.append(rawDataID)

In [5]:
def get_model_classes(src):
    filenames = os.listdir(src)

    for filename in filenames:
        if filename != '.DS_Store':
            if os.path.splitext(filename)[1] == '.json':
                get_model_class(os.path.join(src, filename))

In [6]:
start = time.time()
get_model_classes('/Users/okchuri/project/codestates/CP2/YOLO_Data/validation/annotations/')
print("time :", time.time() - start)

time : 8.469452142715454


In [7]:
len(model_classes)

485

In [8]:
!pwd

/Users/okchuri/project/codestates/CP2


In [21]:
def make_model_label(src, dst):
    with open(src, 'r') as jess:
        jess_dict = json.load(jess)
    
    rawDataID = jess_dict['rawDataInfo']['rawDataID'][9:21]
    resolution = jess_dict['rawDataInfo']['resolution']
    width, height = int(resolution.split('*')[0]), int(resolution.split('*')[1])
    
    with open(dst, 'w') as f:
        for obj in jess_dict['learningDataInfo']['objects']:
            cx = (int(obj['coords']['tl']['x']) + int(obj['coords']['tr']['x'])) / 2 / width
            cy = (int(obj['coords']['tl']['y']) + int(obj['coords']['bl']['y'])) / 2 / height
            w = (int(obj['coords']['tr']['x']) - int(obj['coords']['tl']['x'])) / width
            h = (int(obj['coords']['bl']['y']) - int(obj['coords']['tl']['y'])) / height
            
            txt = str(model_classes.index(rawDataID)) + ' ' + str(cx) + ' ' + str(cy) + ' ' + str(w) + ' ' + str(h) + '\n'
            
            f.write(txt)
            
def make_model_labels(src, dst):
    filenames = os.listdir(src)

    for filename in filenames:
        if filename != '.DS_Store':
            if os.path.splitext(filename)[1] == '.json':
                make_model_label(os.path.join(src, filename), dst+os.path.splitext(filename)[0]+'.txt')

In [23]:
start = time.time()
make_model_labels('/Users/okchuri/project/codestates/CP2/YOLO_Data/train/annotations/', '/Users/okchuri/project/codestates/CP2/YOLO_Data/train/model_labels/')
print("time :", time.time() - start)

time : 155.56005573272705


In [22]:
start = time.time()
make_model_labels('/Users/okchuri/project/codestates/CP2/YOLO_Data/val/annotations/', '/Users/okchuri/project/codestates/CP2/YOLO_Data/val/model_labels/')
print("time :", time.time() - start)

time : 22.781230926513672


In [15]:
yaml_file = 'car_model.yaml'

yaml_data = dict(
    path = "data",
    train = "train",
    val = "val",
    nc = len(model_classes),
    names = model_classes
)

with open(yaml_file, 'w') as f:
    yaml.dump(yaml_data, f, explicit_start = True, default_flow_style = False)

In [26]:
start_time = timer()

!cd ./yolov5 && python train.py --workers 4 --img 640 --batch 64 --epochs 1 --data "data/car_model.yaml" --weights yolov5s.pt --cache

end_time = timer()

print(f'Training time: {(end_time - start_time):.2f}')

train: weights=yolov5s.pt, cfg=, data=data/car_model.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=1, batch_size=64, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=4, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.2-266-g72cad39 Python-3.8.13 torch-1.13.0 CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, translate=0

In [2]:
start_time = timer()

!cd ./yolov5 && python train.py --workers 4 --img 640 --batch 64 --epochs 1 --data "data/car_model.yaml" --weights yolov5s.pt

end_time = timer()

print(f'Training time: {(end_time - start_time):.2f}')

train: weights=yolov5s.pt, cfg=, data=data/car_model.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=1, batch_size=64, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=4, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 7 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v6.2-266-g72cad39 Python-3.8.13 torch-1.13.0 CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0

Training time: 116.70
